#About
## In this notebook we use special classifier which is able to fight correlation with mass

Description of how such classifiers work:
* http://arxiv.org/abs/1305.7248
* http://arxiv.org/abs/1410.4140

In [17]:
import pandas
import evaluation
from hep_ml.gradientboosting import UGradientBoostingClassifier
from hep_ml.losses import BinFlatnessLossFunction

# Read training data

In [11]:
folder = 'tau_data/'
train = pandas.read_csv(folder + 'training.csv', index_col='id')

In [25]:
train.head()

,LifeTime,dira,FlightDistance,FlightDistanceError,IP,IPSig,VertexChi2,pt,DOCAone,DOCAtwo,...,p1_p,p2_p,p0_eta,p1_eta,p2_eta,SPDhits,production,signal,mass,min_ANNmuon
id,,,,,,,,,,,,,,,,,,,,,
18453471,0.001578,0.999999,14.033335,0.681401,0.016039,0.451886,1.900433,1482.037476,0.066667,0.060602,...,12290.760742,39264.398438,3.076006,4.003800,4.031514,458,-99,0,1866.300049,0.277559
5364094,0.000988,0.999705,5.536157,0.302341,0.142163,9.564503,0.865666,3050.720703,0.024022,0.019245,...,16562.667969,7341.257812,3.228553,2.786543,2.975564,406,-99,0,1727.095947,0.225924
11130990,0.000877,0.999984,6.117302,0.276463,0.034746,1.970751,10.975849,3895.908691,0.055044,0.047947,...,22695.388672,10225.309570,3.536903,2.865686,3.052810,196,-99,0,1898.588013,0.368630
15173787,0.000854,0.999903,5.228067,0.220739,0.076389,4.271331,3.276358,4010.781738,0.053779,0.006417,...,16909.515625,9141.426758,3.087461,3.218034,2.375592,137,-99,0,1840.410034,0.246045
1102544,0.001129,0.999995,39.069534,1.898197,0.120936,4.984982,0.468348,4144.546875,0.004491,0.037326,...,97612.804688,47118.785156,4.632295,4.711155,4.296878,477,-99,0,1899.793945,0.222060


In [32]:
train.columns[:-3]

Index([u'LifeTime', u'dira', u'FlightDistance', u'FlightDistanceError', u'IP',
       u'IPSig', u'VertexChi2', u'pt', u'DOCAone', u'DOCAtwo', u'DOCAthree',
       u'IP_p0p2', u'IP_p1p2', u'isolationa', u'isolationb', u'isolationc',
       u'isolationd', u'isolatione', u'isolationf', u'iso', u'CDF1', u'CDF2',
       u'CDF3', u'ISO_SumBDT', u'p0_IsoBDT', u'p1_IsoBDT', u'p2_IsoBDT',
       u'p0_track_Chi2Dof', u'p1_track_Chi2Dof', u'p2_track_Chi2Dof', u'p0_IP',
       u'p1_IP', u'p2_IP', u'p0_IPSig', u'p1_IPSig', u'p2_IPSig', u'p0_pt',
       u'p1_pt', u'p2_pt', u'p0_p', u'p1_p', u'p2_p', u'p0_eta', u'p1_eta',
       u'p2_eta', u'SPDhits', u'production'],
      dtype='object')

In [41]:
type(train.columns[:-4].tolist())

list

In [42]:
variables = train.columns[:-4].tolist()

# Define training features

In [13]:
variables = ['LifeTime',
             'FlightDistance',
             'pt',
             ]

# Training gradient boosting + FlatnessLoss
note, that in flatness loss we point 
* uniform_variables to be `mass`
* uniform_label = 0

not to get correlation between prediction and mass in background. 

In [43]:
loss = BinFlatnessLossFunction(['mass'], n_bins=15, fl_coefficient=0.1, uniform_label=0)
baseline = UGradientBoostingClassifier(loss=loss, n_estimators=40, subsample=0.1, 
                                       max_depth=7, min_samples_leaf=10,
                                       learning_rate=0.1, train_features=variables, random_state=11)
baseline.fit(train[variables + ['mass']], train['signal'])

UGradientBoostingClassifier(learning_rate=0.1,
              loss=BinFlatnessLossFunction(allow_wrong_signs=True, fl_coefficient=0.1, n_bins=15,
            power=2.0, uniform_features=['mass'], uniform_label=array([0])),
              max_depth=7, max_features=None, max_leaf_nodes=None,
              min_samples_leaf=10, min_samples_split=2, n_estimators=40,
              random_state=<mtrand.RandomState object at 0x116b60dc0>,
              splitter='best', subsample=0.1,
              train_features=['LifeTime', 'dira', 'FlightDistance', 'FlightDistanceError', 'IP', 'IPSig', 'VertexChi2', 'pt', 'DOCAone', 'DOCAtwo', 'DOCAthree', 'IP_p0p2', 'IP_p1p2', 'isolationa', 'isolationb', 'isolationc', 'isolationd', 'isolatione', 'isolationf', 'iso', 'CDF1', 'CDF2', 'CDF3', 'ISO_SumBDT', 'p..._IPSig', 'p0_pt', 'p1_pt', 'p2_pt', 'p0_p', 'p1_p', 'p2_p', 'p0_eta', 'p1_eta', 'p2_eta', 'SPDhits'],
              update_tree=True)

# Check agreement test

In [44]:
check_agreement = pandas.read_csv(folder + 'check_agreement.csv', index_col='id')
agreement_probs = baseline.predict_proba(check_agreement[variables])[:, 1]

ks = evaluation.compute_ks(
    agreement_probs[check_agreement['signal'].values == 0],
    agreement_probs[check_agreement['signal'].values == 1],
    check_agreement[check_agreement['signal'] == 0]['weight'].values,
    check_agreement[check_agreement['signal'] == 1]['weight'].values)
print 'KS metric', ks, ks < 0.09

KS metric 0.16845710605820363 False


# Check correlation test

In [45]:
check_correlation = pandas.read_csv(folder + 'check_correlation.csv', index_col='id')
correlation_probs = baseline.predict_proba(check_correlation[variables])[:, 1]
cvm = evaluation.compute_cvm(correlation_probs, check_correlation['mass'])
print 'CvM metric', cvm, cvm < 0.002

CvM metric 0.0010355822379195803 True


# Compute weighted AUC on the training data with min_ANNmuon > 0.4

In [46]:
train_eval = train[train['min_ANNmuon'] > 0.4]
train_probs = baseline.predict_proba(train_eval[variables])[:, 1]
AUC = evaluation.roc_auc_truncated(train_eval['signal'], train_probs)
print 'AUC', AUC

AUC 0.9873945985766308


/Users/wladyslawsurala/repos/flavours-of-physics-start/flavourstaon/lib/python2.7/site-packages/sklearn/metrics/ranking.py:104: DeprecationWarning: The 'reorder' parameter has been deprecated in version 0.20 and will be removed in 0.22. It is recommended not to set 'reorder' and ensure that x is monotonic increasing or monotonic decreasing.
  DeprecationWarning)
/Users/wladyslawsurala/repos/flavours-of-physics-start/flavourstaon/lib/python2.7/site-packages/sklearn/metrics/ranking.py:104: DeprecationWarning: The 'reorder' parameter has been deprecated in version 0.20 and will be removed in 0.22. It is recommended not to set 'reorder' and ensure that x is monotonic increasing or monotonic decreasing.
  DeprecationWarning)
/Users/wladyslawsurala/repos/flavours-of-physics-start/flavourstaon/lib/python2.7/site-packages/sklearn/metrics/ranking.py:104: DeprecationWarning: The 'reorder' parameter has been deprecated in version 0.20 and will be removed in 0.22. It is recommended not to set 'reo

# Predict test, create file for kaggle

In [9]:
test = pandas.read_csv(folder + 'test.csv', index_col='id')
result = pandas.DataFrame({'id': test.index})
result['prediction'] = baseline.predict_proba(test[variables])[:, 1]

In [10]:
result.to_csv('baseline_flatness.csv', index=False, sep=',')